In [1]:
%matplotlib inline
import numpy as np
import nibabel as nib
import matplotlib.pyplot as plt
from dipy.denoise.noise_estimate import estimate_sigma
from dipy.denoise.non_local_means import non_local_means
from os.path import join
from dipy.core.gradients import gradient_table
from dipy.io import read_bvals_bvecs
import dipy.reconst.dti as dti
from pylabs.utils.paths import getnetworkdataroot
fs = getnetworkdataroot()
#set up files post eddy
project = 'bbc'
subjid = [209]
fname_templ = 'sub-bbc{sid}_ses-{snum}_{meth}_{runnum}'
sespassqc = [1]
methodpassqc = ['dti_15dir_b1000']
runpassqc = [1]
dwi_fnames = [fname_templ.format(sid=str(s), snum=str(ses), meth=m, runnum=str(r)) for s, ses, m, r in \
              zip(subjid, sespassqc, methodpassqc, runpassqc)]
#add for loop over dwi_fnames here:
dwi_fname = dwi_fnames[0]
fpath = join(fs, project, dwi_fname.split('_')[0] , dwi_fname.split('_')[1], 'dwi')
fdwi = join(fpath, dwi_fname + '.nii')
fbvecs = join(fpath, dwi_fname + '.bvecs')
fbvals = join(fpath, dwi_fname + '.bvals')
fbvecs_ec = join(fpath, dwi_fname + '_eddy_corrected.eddy_rotated_bvecs')
S0_fname = join(fpath, dwi_fname + '_S0.nii')
brain_outfname = S0_fname[: -6] + 'S0_brain'
#make gtab
bvals_ec, bvecs_ec = read_bvals_bvecs(fbvals, fbvecs_ec)
gtab_ec = gradient_table(bvals_ec, bvecs_ec)
#open and denoise ec data
img_ec = nib.load(join(fpath, fdwi[:-4] + '_eddy_corrected.nii.gz'))
data_ec = img_ec.get_data()
mask_img = nib.load(brain_outfname + '_mask.nii')
mask = mask_img.get_data()
sigma4 = estimate_sigma(data_ec, N=4)
den4_data_ec = non_local_means(data_ec, sigma=sigma4, mask=mask)
sigma8 = estimate_sigma(data_ec, N=8)
den8_data_ec = non_local_means(data_ec, sigma=sigma8, mask=mask)
#save denoised images
den4_data_ec_img = nib.nifti1.Nifti1Image(den4_data_ec, img_ec.affine)
den4_data_ec_img.set_qform(img_ec.affine, code=1)
nib.save(den4_data_ec_img, join(fpath, dwi_fname + '_ec_den4_dipy.nii'))
den8_data_ec_img = nib.nifti1.Nifti1Image(den8_data_ec, img_ec.affine)
den8_data_ec_img.set_qform(img_ec.affine, code=1)
nib.save(den8_data_ec_img, join(fpath, dwi_fname + '_ec_den8_dipy.nii'))
#now we have 3 raw files eddy output, den4 and den4

In [7]:
%%bash
meth=("" -w)
tens=(_eddy_corrected.nii _ec_den4_dipy.nii _ec_den8_dipy.nii _eddy_corrected_restore_tensor_den8.nii _eddy_corrected_restore_tensor.nii)
rfn=/media/DiskArray/shared_data/js/bbc/sub-bbc209/ses-1/dwi/sub-bbc209_ses-1_dti_15dir_b1000_1
bvals=${rfn}.bvals
bvecs=${rfn}_eddy_corrected.eddy_rotated_bvecs
brm=${rfn}_S0_brain_mask.nii
mas=(${rfn}_S0_brain-maskCC.nii.gz ${rfn}_S0_brain-maskCO.nii.gz ${rfn}_S0_brain-maskSL.nii.gz)
S0=${rfn}_S0.nii
rm -f ${rfn:0: -34}tensor_stats.txt ${rfn:0: -34}tmpresult.txt ${rfn:0: -34}tmpstat.txt
touch ${rfn:0: -34}tensor_stats.txt
for d in "${tens[@]}"; do
    for m in "${meth[@]}"; do
        if [[ ${d} == '_eddy_corrected.nii' ]] ||  [[ ${d} == '_ec_den4_dipy.nii' ]] || [[ ${d} == '_ec_den8_dipy.nii' ]]; then
            dtifit -k ${rfn}${d} -o ${rfn}${d:0: -4}`if [[ "${m}" == '-w' ]]; then echo _wls; else echo _ols; fi` \
                -m $brm -r $bvecs -b $bvals $m --save_tensor --sse
            dtigen -t ${rfn}${d:0: -4}`if [[ "${m}" == '-w' ]];  then echo _wls; else echo _ols; fi`_tensor \
                -o ${rfn}${d:0: -4}`if [[ "${m}" == '-w' ]];  then echo _wls; else echo _ols; fi`_dtigen \
                -b $bvals -r $bvecs -m $brm --s0=${S0}
            fslmaths ${rfn}${d:0: -4}`if [[ "${m}" == '-w' ]];  then echo _wls; else echo _ols; fi`_dtigen -thr 5 -uthr 10000 \
                -sub ${rfn}${d} ${rfn}${d:0: -4}`if [[ "${m}" == '-w' ]];  then echo _wls; else echo _ols; fi`_dtigen_sub${d:0: -4}
        elif [[ ${m} == '-w' ]]; then
            dtigen -t ${rfn}${d:0: -4} -o ${rfn}${d:0: -4}_dtigen -b $bvals -r $bvecs -m $brm --s0=${S0}
            if [[ ${d} == '_eddy_corrected_restore_tensor.nii' ]];  then
                fslmaths ${rfn}${d:0: -4}_dtigen -thr 5 -uthr 10000 -sub ${rfn}_eddy_corrected.nii ${rfn}${d:0: -4}_dtigen_sub_eddy_corrected
            else
                fslmaths ${rfn}${d:0: -4}_dtigen -thr 5 -uthr 10000 -sub ${rfn}_ec_den8_dipy.nii ${rfn}${d:0: -4}_dtigen_sub_ec_den8_dipy
            fi
        fi
        for s in "${mas[@]}"; do
            if [[ ${d} == '_eddy_corrected.nii' ]] ||  [[ ${d} == '_ec_den4_dipy.nii' ]] || [[ ${d} == '_ec_den8_dipy.nii' ]];  then
                echo ${d:0: -4}`if [[ "${m}" == '-w' ]];  then echo _wls; else echo _ols; fi`_dtigen_sub${d:0: -4}_${s: -9: -7} \
                    > ${rfn:0: -34}tmpstat.txt
                ls -l $s
                fslstats -t ${rfn}${d:0: -4}`if [[ "${m}" == '-w' ]];  then echo _wls; else echo _ols; fi`_dtigen_sub${d:0: -4} \
                    -k $s -M >> ${rfn:0: -34}tmpstat.txt
            elif [[ ${d} == '_eddy_corrected_restore_tensor.nii' ]] && [[ ${m} == '-w' ]];  then
                echo ${d:0: -4}_dtigen_sub_eddy_corrected_${s: -9: -7} > ${rfn:0: -34}tmpstat.txt
                fslstats -t ${rfn}${d:0: -4}_dtigen_sub_eddy_corrected -k $s -M >> ${rfn:0: -34}tmpstat.txt
            elif [[ ${d} == '_eddy_corrected_restore_tensor_den8.nii' ]] && [[ ${m} == '-w' ]];  then
                echo ${d:0: -4}_dtigen_sub_ec_den8_dipy_${s: -9: -7} > ${rfn:0: -34}tmpstat.txt
                fslstats -t ${rfn}${d:0: -4}_dtigen_sub_ec_den8_dipy -k $s -M >> ${rfn:0: -34}tmpstat.txt
            fi
            cat ${rfn:0: -34}tmpstat.txt
            paste ${rfn:0: -34}tensor_stats.txt ${rfn:0: -34}tmpstat.txt > ${rfn:0: -34}tmpresult.txt
            mv ${rfn:0: -34}tmpresult.txt ${rfn:0: -34}tensor_stats.txt
            echo finished $d $m $s
        done
    done
done
rm -f ${rfn:0: -34}tmpresult.txt ${rfn:0: -34}tmpstat.txt

0 144 0 144 0 64
0 slices processed
1 slices processed
2 slices processed
3 slices processed
4 slices processed
5 slices processed
6 slices processed
7 slices processed
8 slices processed
9 slices processed
10 slices processed
11 slices processed
12 slices processed
13 slices processed
14 slices processed
15 slices processed
16 slices processed
17 slices processed
18 slices processed
19 slices processed
20 slices processed
21 slices processed
22 slices processed
23 slices processed
24 slices processed
25 slices processed
26 slices processed
27 slices processed
28 slices processed
29 slices processed
30 slices processed
31 slices processed
32 slices processed
33 slices processed
34 slices processed
35 slices processed
36 slices processed
37 slices processed
38 slices processed
39 slices processed
40 slices processed
41 slices processed
42 slices processed
43 slices processed
44 slices processed
45 slices processed
46 slices processed
47 slices processed
48 slices processed
49 slices pro

rm: invalid option -- '3'
Try 'rm --help' for more information.
ERROR:: Empty mask image
ERROR:: Empty mask image
rm: invalid option -- '3'
Try 'rm --help' for more information.
